# Scraping TripAdvisor with Selenium 2020

In [ ]:
#based on code by giuseppegambino https://github.com/giuseppegambino/Scraping-TripAdvisor-with-Python-2020/

In this notebook It is described how to scrape TripAdvisor reviews with Selenium.

The code is commented but if you have any doubts, don't hesitate to get in touch with me

## Step 1: import the libraries 

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [ ]:
from selenium.webdriver.common.keys import Keys

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from selenium.webdriver.chrome.options import Options

In [ ]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

## Step 2: load the web driver and the page

In [ ]:
# function to check if the button is on the page, to avoid miss-click problem
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [ ]:
#from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
#driver = webdriver.Chrome(ChromeDriverManager().install()) - run if not the driver is not installed

In [ ]:
#options = Options()
#options.headless = False
driver = webdriver.Chrome('/Users/HIPark/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver') #directory where the folder is installed
driver.minimize_window()
#main search page
driver.get('https://www.tripadvisor.com/Hotels-g60745-Boston_Massachusetts-Hotels.html') #change url according to the city

In [ ]:
#check the 3 stars checkbox
driver.find_elements_by_xpath(".//div[@class='ui_checkbox u-658Xev _1kVF4wUw']")[22].click()
time.sleep(5)

In [ ]:
#get url of all hotels in the first two pages
check = []
hotel_dict = {}
for l in range(2):
    #find all hotels in the page
    hotels = driver.find_elements_by_xpath(".//div[@class='meta_listing ui_columns large_thumbnail_mobile ']")
    for j in tqdm(range(len(hotels))):
        try:
            name = hotels[j].find_elements_by_xpath(".//a[@data-clicksource='HotelName']")[0].text
            link = hotels[j].find_element_by_css_selector('a').get_attribute('href')
            hotel_dict[name] = link 
            check.append(name)
        except:
            pass
    #if (check_exists_by_xpath('//a[@class="nav next ui_button primary cx_brand_refresh_phase2"]')):
    driver.find_element_by_xpath('//a[@class="nav next ui_button primary  cx_brand_refresh_phase2"]').click()
    time.sleep(5)
driver.close()

In [ ]:
hotel_lst = [(k,v) for k, v in hotel_dict.items()]

In [ ]:
lst = []

In [ ]:
#go through all hotel urls and get the ratings
current_len = len(lst)
driver = webdriver.Chrome('/Users/HIPark/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver')
driver.minimize_window()
for i, name_link in tqdm(enumerate(hotel_lst[0:])):
    name, link = name_link
    print(i, name)
    # go to hotel page
    driver.get(link)
    #hotels[j].find_elements_by_xpath(".//div[@class='ui_column is-narrow title_wrap']")[0].click()
    #driver.switch_to.window(driver.window_handles[-1])
    #driver.minimize_window()

    #Only pick ratings 1~3
    time.sleep(10)
    driver.find_elements_by_xpath('.//label[@class="_2PPG44IR _1o34NnSP"]')[2].click()
    time.sleep(5)
    driver.find_elements_by_xpath('.//label[@class="_2PPG44IR _1o34NnSP"]')[3].click()
    time.sleep(5)
    driver.find_elements_by_xpath('.//label[@class="_2PPG44IR _1o34NnSP"]')[4].click()
    time.sleep(5)
    driver.minimize_window()

    for i in tqdm(range(20)): #first 20 pages of review

        try:
            # to expand the review 
            driver.find_element_by_xpath(".//div[@class='XUVJZtom']").click()
            time.sleep(5)
        except:
            print('Cant expand')
            pass

        container = driver.find_elements_by_xpath(".//div[@class='_2wrUUKlw _3hFEdNs8']")
        num_page_items = len(container)

        for k in range(num_page_items):
            try:
                response = container[k].find_element_by_xpath(".//span[@class='sT5TMxg3']").text.replace("\n", "")
            except:
                continue

            # to save the rating
            string = container[k].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")
            data = string.split("_")
            if int(data[3]) > 30:
                continue
            review = container[k].find_element_by_xpath(".//q[@class='IRsGHoPm']").text.replace("\n", "")

            try:
                trip_type = container[k].find_elements_by_xpath(".//span[@class='_2bVY3aT5']")[0].text
            except:
                trip_type = None

            try:
                date_of_stay = container[k].find_elements_by_xpath(".//span[@class='_34Xs-BQm']")[0].text
            except:
                trip_type = None
            
            try:
                manager, response_date = tuple(container[k].find_elements_by_xpath(".//div[@class='_204cKjWJ']")[0].text.split('\n'))
            except:
                manager = response_date = container[k].find_elements_by_xpath(".//div[@class='_204cKjWJ']")[0].text

            # to save in a csv file readable the star and the review [Ex: 50,"I love this place"]
            
            lst.append([name, data[3], review, response, trip_type, date_of_stay, manager, response_date])

        # to change the page
        if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')):
            driver.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
        else:
            print('Next button does not exist')
            break
        time.sleep(5)
    
    print('Collected', len(lst) - current_len, 'Responses')
    current_len = len(lst)
    #driver.close()
    #driver.switch_to.window(driver.window_handles[0])
    driver.minimize_window()
driver.close()

## Step 3: Save reviews into csv

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(lst)
df.columns = ['Hotel', 'Rating', 'Review', 'Response', 'Trip type', 'Date of Stay', 'Responder', 'Responded date']


In [ ]:
df.to_csv('boston.csv') #change name according to the city